<a href="https://colab.research.google.com/github/k1151msarandega/1st-order/blob/main/1st_order_phase_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Architecture:** *1st-order model*

**Encoding Scheme:** *Phase encoding*

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from snntorch import encoding
import time

1. Set the device to GPU if available, otherwise CPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. Define the Spiking Neural Network (SNN) model

In [ ]:
class SNNModel(nn.Module):
    def __init__(self):
        super(SNNModel, self).__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = encoding.phase(x, duration=32)
        x = self.fc1(x)
        x = encoding.phase(x, duration=32)
        x = self.fc2(x)
        return x

3. Define the training and testing functions

In [ ]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Forward pass
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    end_time = time.time()
    accuracy = 100 * correct / total
    test_time = end_time - start_time
    return accuracy, test_time

4. Set the hyperparameters

In [ ]:
batch_size = 128
learning_rate = 0.001
num_epochs = 10

5. Load the MNIST dataset and apply transformations

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

6. Create an instance of the SNN model and move it to the device

In [ ]:
model = SNNModel().to(device)

7. Define the loss function and optimiser

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


8. Training loop

In [ ]:
start_time = time.time()
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer)
    accuracy, _ = test(model, test_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%")

end_time = time.time()
total_time = end_time - start_time
print(f"Total training time: {total_time:.2f} seconds")

9. Testing phase

In [ ]:
_, test_time = test(model, test_loader)
print(f"Testing time: {test_time:.2f} seconds")